In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

In [ ]:
def knncls():
    """
    KNN预测用户标签
    """
    # 读取数据
    data = pd.read_csv("D:/Data/facebookdata/train.csv")
    print(data.head(10))
    
    # 处理数据
    # 1.缩小数据, query输入字符串, 限定条件
    data = data.query("x > 1.0 & x < 1.25 & y>2.5 & y < 2.75") 
    # 处理时间, to_datatime, 里面是要转换的特征
    time_value = pd.to_datetime(data['time'], unit='s')
    # print(time_value.head(10))
    
    # 把日期格式转换为字典格式
    time_value = pd.DatetimeIndex(time_value)
    
    # 构造一些特征
    #data['day'] = time_value.day
    #data['hour'] = time_value.hour
    #data['weekday'] = time_value.weekday
    
    # loc会直接将数值放进去, 而不是复制进去
    data.loc[:,'day'] = time_value.day
    data.loc[:,'hour'] = time_value.hour
    # data.loc[:,'weekday'] = time_value.weekday
    
    # 把时间戳删除 (pandas的列是1, sk是0)
    data = data.drop(['time'],axis=1)
    # print(data.head(5))
    
    # 把签到数量少于N个的删除
    place_count = data.groupby('place_id').count()
    # reset_index 将某个特征变成索引
    tf = place_count[place_count.row_id>3].reset_index()
    data = data[data['place_id'].isin(tf.place_id)]
    # 去除特征的特征值和目标值
    y = data['place_id']
    x = data.drop(['place_id'], axis=1)
    x = data.drop(['row_id'], axis=1)
    
    # 数据分割, train和test
    x_train,x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)
    
    
    # 特征工程(标准化)
    std = StandardScaler()
    # 对train & test standard
    x_train = std.fit_transform(x_train)
    x_test =  std.fit_transform(x_test)
    
    # 进行算法
    knn = KNeighborsClassifier(n_neighbors=5)
    # 训练 
    knn.fit(x_train, y_train)
    # 得出结果
    y_predict = knn.predict(x_test)
    print("预测的目标签到位置:", y_predict)
    
    # 得到准确率
    print("准确率 : ", knn.score(x_test, y_test))
    
    return None

In [ ]:
if __name__ == "__main__":
    knncls()